# MINOTAUR Optimization Workflow

This notebook demonstrates using MINOTAUR for design space exploration and optimization.

In [ ]:
import minotaur
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## 1. Design Space Exploration

Generate a high-resolution parameter sweep to understand the design space.

In [ ]:
# High-resolution sweep
bpr_range = np.linspace(0.2, 1.5, 31)
opr_range = np.linspace(4.0, 16.0, 31)

results = minotaur.sweep(bpr_range, opr_range, mach=0.65, alt_km=8.0)

n_bpr = results['n_bpr']
n_opr = results['n_opr']

BPR = results['bpr'].reshape(n_opr, n_bpr)
OPR = results['opr'].reshape(n_opr, n_bpr)
TSFC = results['tsfc'].reshape(n_opr, n_bpr)
THRUST = results['thrust'].reshape(n_opr, n_bpr)
T4 = results['t4'].reshape(n_opr, n_bpr)
STATUS = results['status'].reshape(n_opr, n_bpr)

# Mask invalid points
valid = STATUS == minotaur.STATUS_OK
TSFC_valid = np.where(valid, TSFC, np.nan)
THRUST_valid = np.where(valid, THRUST, np.nan)
T4_valid = np.where(valid, T4, np.nan)

print(f"Valid design points: {np.sum(valid)}/{valid.size} ({100*np.sum(valid)/valid.size:.1f}%)")

In [ ]:
# 3D surface plots
fig = plt.figure(figsize=(14, 5))

# TSFC surface
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(BPR, OPR, TSFC_valid, cmap=cm.viridis, alpha=0.8)
ax1.set_xlabel('BPR')
ax1.set_ylabel('OPR')
ax1.set_zlabel('TSFC')
ax1.set_title('TSFC Surface')

# Thrust surface
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(BPR, OPR, THRUST_valid, cmap=cm.plasma, alpha=0.8)
ax2.set_xlabel('BPR')
ax2.set_ylabel('OPR')
ax2.set_zlabel('Thrust')
ax2.set_title('Thrust Surface')

# T4 surface
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(BPR, OPR, T4_valid, cmap=cm.inferno, alpha=0.8)
ax3.set_xlabel('BPR')
ax3.set_ylabel('OPR')
ax3.set_zlabel('T4 [K]')
ax3.set_title('T4 Surface')

plt.tight_layout()
plt.show()

## 2. Feasibility Region Analysis

Identify regions where the solver converges and constraints are satisfied.

In [ ]:
# Identify different failure modes
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Convergence map
ax1 = axes[0]
convergence_map = np.zeros_like(STATUS, dtype=float)
convergence_map[STATUS == minotaur.STATUS_OK] = 1.0
convergence_map[STATUS == minotaur.STATUS_MAXITER] = 0.5
convergence_map[STATUS == minotaur.STATUS_CONSTRAINT_VIOL] = 0.25

im1 = ax1.imshow(convergence_map, extent=[bpr_range[0], bpr_range[-1], opr_range[0], opr_range[-1]],
                 origin='lower', aspect='auto', cmap='RdYlGn')
ax1.set_xlabel('Bypass Ratio (BPR)')
ax1.set_ylabel('Overall Pressure Ratio (OPR)')
ax1.set_title('Feasibility Map')
cbar1 = plt.colorbar(im1, ax=ax1, ticks=[0.25, 0.5, 1.0])
cbar1.ax.set_yticklabels(['Constraint Viol', 'Max Iter', 'Converged'])

# T4 constraint visualization
ax2 = axes[1]
t4_limit = 1400.0
t4_margin = (t4_limit - T4_valid) / t4_limit * 100  # Margin as percentage

im2 = ax2.contourf(BPR, OPR, t4_margin, levels=np.linspace(-10, 20, 31), cmap='RdYlBu')
ax2.contour(BPR, OPR, t4_margin, levels=[0], colors='k', linewidths=2)  # T4 = T4_max line
ax2.set_xlabel('Bypass Ratio (BPR)')
ax2.set_ylabel('Overall Pressure Ratio (OPR)')
ax2.set_title(f'T4 Margin (T4_max = {t4_limit} K)')
cbar2 = plt.colorbar(im2, ax=ax2)
cbar2.set_label('Margin [%]')

plt.tight_layout()
plt.show()

## 3. Trade-off Analysis

Visualize the trade-off between TSFC and Thrust (Pareto-like frontier).

In [ ]:
# Flatten valid points
tsfc_flat = TSFC_valid.flatten()
thrust_flat = THRUST_valid.flatten()
bpr_flat = BPR.flatten()
opr_flat = OPR.flatten()
t4_flat = T4_valid.flatten()

# Remove NaN
mask = ~np.isnan(tsfc_flat) & ~np.isnan(thrust_flat)
tsfc_valid = tsfc_flat[mask]
thrust_valid = thrust_flat[mask]
bpr_valid = bpr_flat[mask]
opr_valid = opr_flat[mask]
t4_valid_flat = t4_flat[mask]

# Find Pareto-optimal points (minimize TSFC, maximize Thrust)
def find_pareto_front(tsfc, thrust):
    """Find Pareto-optimal points (minimize TSFC, maximize thrust)."""
    n = len(tsfc)
    pareto = np.ones(n, dtype=bool)
    for i in range(n):
        for j in range(n):
            if i != j:
                # j dominates i if j has lower TSFC and higher thrust
                if tsfc[j] <= tsfc[i] and thrust[j] >= thrust[i]:
                    if tsfc[j] < tsfc[i] or thrust[j] > thrust[i]:
                        pareto[i] = False
                        break
    return pareto

pareto_mask = find_pareto_front(tsfc_valid, thrust_valid)
print(f"Pareto-optimal points: {np.sum(pareto_mask)}/{len(tsfc_valid)}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# TSFC vs Thrust scatter
ax1 = axes[0]
sc = ax1.scatter(tsfc_valid[~pareto_mask], thrust_valid[~pareto_mask], 
                 c=t4_valid_flat[~pareto_mask], cmap='plasma', alpha=0.5, s=20)
ax1.scatter(tsfc_valid[pareto_mask], thrust_valid[pareto_mask], 
            c='red', s=50, marker='*', label='Pareto Front', zorder=5)

# Sort and connect Pareto points
pareto_idx = np.argsort(tsfc_valid[pareto_mask])
ax1.plot(tsfc_valid[pareto_mask][pareto_idx], thrust_valid[pareto_mask][pareto_idx], 
         'r-', linewidth=2, alpha=0.7)

ax1.set_xlabel('TSFC Proxy (minimize)')
ax1.set_ylabel('Thrust Proxy (maximize)')
ax1.set_title('TSFC-Thrust Trade-off')
ax1.legend()
cbar = plt.colorbar(sc, ax=ax1)
cbar.set_label('T4 [K]')

# Pareto front colored by BPR and OPR
ax2 = axes[1]
sc2 = ax2.scatter(tsfc_valid[pareto_mask], thrust_valid[pareto_mask], 
                  c=bpr_valid[pareto_mask], cmap='coolwarm', s=100, marker='o')
for i, (t, th, b, o) in enumerate(zip(tsfc_valid[pareto_mask], thrust_valid[pareto_mask], 
                                       bpr_valid[pareto_mask], opr_valid[pareto_mask])):
    if i % 3 == 0:  # Label every 3rd point
        ax2.annotate(f'({b:.2f},{o:.1f})', (t, th), fontsize=7, 
                     xytext=(5, 5), textcoords='offset points')

ax2.set_xlabel('TSFC Proxy')
ax2.set_ylabel('Thrust Proxy')
ax2.set_title('Pareto Front Design Points (BPR, OPR)')
cbar2 = plt.colorbar(sc2, ax=ax2)
cbar2.set_label('BPR')

plt.tight_layout()
plt.show()

## 4. Gradient-Based Optimization

Use sensitivity information for gradient-based optimization.

In [ ]:
def objective_tsfc(x):
    """Objective: minimize TSFC."""
    bpr, opr = x
    result = minotaur.solve(mach=0.65, alt_km=8.0, bpr=bpr, opr=opr)
    if result.converged:
        return result.tsfc_proxy
    return np.inf

def gradient_tsfc(x, h=1e-4):
    """Finite difference gradient for TSFC."""
    bpr, opr = x
    f0 = objective_tsfc(x)
    
    grad = np.zeros(2)
    for i in range(2):
        x_plus = x.copy()
        x_plus[i] += h
        grad[i] = (objective_tsfc(x_plus) - f0) / h
    
    return grad

# Simple gradient descent
def gradient_descent(x0, lr=0.01, max_iter=50, tol=1e-6):
    """Simple gradient descent optimizer."""
    x = np.array(x0, dtype=float)
    history = [x.copy()]
    f_history = [objective_tsfc(x)]
    
    for i in range(max_iter):
        grad = gradient_tsfc(x)
        x_new = x - lr * grad
        
        # Bounds: BPR in [0.2, 1.5], OPR in [4, 16]
        x_new[0] = np.clip(x_new[0], 0.2, 1.5)
        x_new[1] = np.clip(x_new[1], 4.0, 16.0)
        
        f_new = objective_tsfc(x_new)
        history.append(x_new.copy())
        f_history.append(f_new)
        
        if np.abs(f_new - f_history[-2]) < tol:
            break
        
        x = x_new
    
    return x, history, f_history

# Run optimization from different starting points
starting_points = [
    [0.4, 6.0],
    [0.8, 10.0],
    [1.2, 12.0]
]

all_histories = []
for x0 in starting_points:
    x_opt, history, f_history = gradient_descent(x0, lr=0.05, max_iter=30)
    all_histories.append((history, f_history))
    print(f"Start: BPR={x0[0]:.2f}, OPR={x0[1]:.1f} -> Opt: BPR={x_opt[0]:.3f}, OPR={x_opt[1]:.2f}, TSFC={f_history[-1]:.4f}")

In [ ]:
# Visualize optimization trajectories
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Contour with trajectories
ax1 = axes[0]
cs = ax1.contourf(BPR, OPR, TSFC_valid, levels=30, cmap='viridis', alpha=0.8)
ax1.contour(BPR, OPR, TSFC_valid, levels=15, colors='k', linewidths=0.3, alpha=0.5)

colors = ['red', 'orange', 'magenta']
for (history, f_history), color, x0 in zip(all_histories, colors, starting_points):
    hist = np.array(history)
    ax1.plot(hist[:, 0], hist[:, 1], 'o-', color=color, markersize=4, linewidth=1.5,
             label=f'Start ({x0[0]:.1f}, {x0[1]:.0f})')
    ax1.plot(hist[0, 0], hist[0, 1], 's', color=color, markersize=10)  # Start
    ax1.plot(hist[-1, 0], hist[-1, 1], '*', color=color, markersize=15)  # End

ax1.set_xlabel('Bypass Ratio (BPR)')
ax1.set_ylabel('Overall Pressure Ratio (OPR)')
ax1.set_title('Optimization Trajectories on TSFC Contour')
ax1.legend(loc='upper right')
plt.colorbar(cs, ax=ax1, label='TSFC')

# Convergence plot
ax2 = axes[1]
for (history, f_history), color, x0 in zip(all_histories, colors, starting_points):
    ax2.plot(f_history, 'o-', color=color, label=f'Start ({x0[0]:.1f}, {x0[1]:.0f})')

ax2.set_xlabel('Iteration')
ax2.set_ylabel('TSFC')
ax2.set_title('Optimization Convergence')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Multi-Altitude Performance Map

Analyze how optimal design changes with altitude.

In [ ]:
# Fixed design point, varying altitude
altitudes = np.linspace(0, 15, 31)
bpr_design = 0.6
opr_design = 8.0

results_alt = []
for alt in altitudes:
    result = minotaur.solve(mach=0.65, alt_km=alt, bpr=bpr_design, opr=opr_design)
    results_alt.append({
        'alt': alt,
        'converged': result.converged,
        'tsfc': result.tsfc_proxy if result.converged else np.nan,
        'thrust': result.thrust_proxy if result.converged else np.nan,
        't4': result.t4 if result.converged else np.nan
    })

alt_arr = np.array([r['alt'] for r in results_alt])
tsfc_arr = np.array([r['tsfc'] for r in results_alt])
thrust_arr = np.array([r['thrust'] for r in results_alt])
t4_arr = np.array([r['t4'] for r in results_alt])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(alt_arr, tsfc_arr, 'b-o', markersize=4)
axes[0].set_xlabel('Altitude [km]')
axes[0].set_ylabel('TSFC Proxy')
axes[0].set_title(f'TSFC vs Altitude (BPR={bpr_design}, OPR={opr_design})')
axes[0].grid(True, alpha=0.3)

axes[1].plot(alt_arr, thrust_arr, 'g-o', markersize=4)
axes[1].set_xlabel('Altitude [km]')
axes[1].set_ylabel('Thrust Proxy')
axes[1].set_title('Thrust vs Altitude')
axes[1].grid(True, alpha=0.3)

axes[2].plot(alt_arr, t4_arr, 'r-o', markersize=4)
axes[2].axhline(y=1400, color='k', linestyle='--', label='T4_max')
axes[2].set_xlabel('Altitude [km]')
axes[2].set_ylabel('T4 [K]')
axes[2].set_title('Turbine Inlet Temperature vs Altitude')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Monte Carlo Uncertainty Quantification

Assess output uncertainty given input parameter uncertainty.

In [ ]:
# Nominal values with uncertainty
bpr_nom = 0.6
opr_nom = 8.0
eta_comp_nom = 0.82
eta_turb_nom = 0.86

# Uncertainty (standard deviation as fraction of nominal)
bpr_std = 0.02 * bpr_nom
opr_std = 0.02 * opr_nom
eta_comp_std = 0.01 * eta_comp_nom
eta_turb_std = 0.01 * eta_turb_nom

n_samples = 500
np.random.seed(42)

mc_results = []
for _ in range(n_samples):
    bpr_sample = np.random.normal(bpr_nom, bpr_std)
    opr_sample = np.random.normal(opr_nom, opr_std)
    eta_c_sample = np.random.normal(eta_comp_nom, eta_comp_std)
    eta_t_sample = np.random.normal(eta_turb_nom, eta_turb_std)
    
    result = minotaur.solve(
        mach=0.65, alt_km=8.0, bpr=bpr_sample, opr=opr_sample,
        eta_comp=eta_c_sample, eta_turb=eta_t_sample
    )
    
    if result.converged:
        mc_results.append({
            'bpr': bpr_sample, 'opr': opr_sample,
            'eta_c': eta_c_sample, 'eta_t': eta_t_sample,
            'tsfc': result.tsfc_proxy,
            'thrust': result.thrust_proxy,
            't4': result.t4
        })

print(f"Converged: {len(mc_results)}/{n_samples} samples")

In [ ]:
tsfc_mc = np.array([r['tsfc'] for r in mc_results])
thrust_mc = np.array([r['thrust'] for r in mc_results])
t4_mc = np.array([r['t4'] for r in mc_results])

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# TSFC histogram
ax1 = axes[0]
ax1.hist(tsfc_mc, bins=30, density=True, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tsfc_mc), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(tsfc_mc):.4f}')
ax1.axvline(np.mean(tsfc_mc) + np.std(tsfc_mc), color='orange', linestyle=':', linewidth=2)
ax1.axvline(np.mean(tsfc_mc) - np.std(tsfc_mc), color='orange', linestyle=':', linewidth=2, 
            label=f'Std: {np.std(tsfc_mc):.4f}')
ax1.set_xlabel('TSFC Proxy')
ax1.set_ylabel('Density')
ax1.set_title('TSFC Distribution')
ax1.legend(fontsize=8)

# Thrust histogram
ax2 = axes[1]
ax2.hist(thrust_mc, bins=30, density=True, color='forestgreen', alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(thrust_mc), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(thrust_mc):.4f}')
ax2.axvline(np.mean(thrust_mc) + np.std(thrust_mc), color='orange', linestyle=':', linewidth=2)
ax2.axvline(np.mean(thrust_mc) - np.std(thrust_mc), color='orange', linestyle=':', linewidth=2,
            label=f'Std: {np.std(thrust_mc):.4f}')
ax2.set_xlabel('Thrust Proxy')
ax2.set_ylabel('Density')
ax2.set_title('Thrust Distribution')
ax2.legend(fontsize=8)

# T4 histogram
ax3 = axes[2]
ax3.hist(t4_mc, bins=30, density=True, color='coral', alpha=0.7, edgecolor='black')
ax3.axvline(np.mean(t4_mc), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(t4_mc):.1f} K')
ax3.axvline(np.mean(t4_mc) + np.std(t4_mc), color='orange', linestyle=':', linewidth=2)
ax3.axvline(np.mean(t4_mc) - np.std(t4_mc), color='orange', linestyle=':', linewidth=2,
            label=f'Std: {np.std(t4_mc):.1f} K')
ax3.axvline(1400, color='black', linestyle='-', linewidth=2, label='T4_max')
ax3.set_xlabel('T4 [K]')
ax3.set_ylabel('Density')
ax3.set_title('T4 Distribution')
ax3.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Summary statistics
print("\nUncertainty Quantification Summary:")
print(f"  TSFC: {np.mean(tsfc_mc):.4f} +/- {np.std(tsfc_mc):.4f} (CV: {100*np.std(tsfc_mc)/np.mean(tsfc_mc):.2f}%)")
print(f"  Thrust: {np.mean(thrust_mc):.4f} +/- {np.std(thrust_mc):.4f} (CV: {100*np.std(thrust_mc)/np.mean(thrust_mc):.2f}%)")
print(f"  T4: {np.mean(t4_mc):.1f} +/- {np.std(t4_mc):.1f} K (CV: {100*np.std(t4_mc)/np.mean(t4_mc):.2f}%)")